In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
DATASET_PATH  = '/kaggle/input/ml100-03-final/image_data/train'
TEST_PATH = '/kaggle/input/ml100-03-final/image_data/test'
OUTPUT_PATH = '/kaggle/working'
NUM_CLASSES = 5
NUM_EPOCHS = 32
SEED = 77
# saved model
WEIGHTS_FINAL = 'model-InceptionResNetV2.h5'
categories=os.listdir(DATASET_PATH)

In [3]:
# Image preprocess
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.125, height_shift_range=0.125, zoom_range=0.125, horizontal_flip=True,
                                   validation_split=0.2, rescale=1. / 255)
train_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'training', seed = SEED)
valid_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'validation', seed = SEED)
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [4]:
# Build Model
net = InceptionResNetV2(include_top=False, input_shape=(256, 256, 3))
x = net.output
x = Flatten()(x)
# Add Dense layer, each probability by softmax
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
net_final.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

219062272/219055592 [==============================] - 3s 0us/step


In [5]:
# Train
net_final.fit_generator(train_batches, validation_data = valid_batches, epochs = NUM_EPOCHS)
# Store Model
net_final.save(WEIGHTS_FINAL)

Epoch 1/32
71/71 [==============================] - 188s 3s/step - loss: 1.5067 - accuracy: 0.7058 - val_loss: 35942.2539 - val_accuracy: 0.2007
Epoch 2/32
71/71 [==============================] - 63s 887ms/step - loss: 1.6534 - accuracy: 0.7438 - val_loss: 445303904.0000 - val_accuracy: 0.2114
Epoch 3/32
71/71 [==============================] - 65s 910ms/step - loss: 1.1558 - accuracy: 0.7252 - val_loss: 492.6474 - val_accuracy: 0.2131
Epoch 4/32
71/71 [==============================] - 64s 906ms/step - loss: 1.3593 - accuracy: 0.7292 - val_loss: 1.7936 - val_accuracy: 0.6590
Epoch 5/32
71/71 [==============================] - 65s 909ms/step - loss: 0.7004 - accuracy: 0.7956 - val_loss: 2.6283 - val_accuracy: 0.7265
Epoch 6/32
71/71 [==============================] - 65s 909ms/step - loss: 0.6609 - accuracy: 0.8119 - val_loss: 0.6273 - val_accuracy: 0.7496
Epoch 7/32
71/71 [==============================] - 65s 916ms/step - loss: 0.4927 - accuracy: 0.8363 - val_loss: 0.5745 - val_accu

In [6]:
from keras.preprocessing import image
out = np.array(['id', 'flower_class'])
testfiles=os.listdir(TEST_PATH)
for testfile in testfiles:
    filename = testfile.split('.')[0]
    img = image.load_img(TEST_PATH+'/'+testfile,target_size=(256, 256))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = x /255.
    pred = net_final.predict(x)[0]
    tof=np.argmax(pred)
    out = np.vstack((out,[filename, tof]))

pd.DataFrame(out).to_csv(OUTPUT_PATH+'/prediction.csv',index=False,header=False)